In [1]:
!pip install roboflow

  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.3 kB)
  Using cached pillow-11.3.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.0 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached PyYAML-6.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached contourpy-1.3.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached charset_normalizer-3.4.3-cp313-cp313-macosx_10_13_universal2.whl.metadata (36 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 24.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.4/642.4 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [2]:
ROBOFLOW_API_KEY = "bqQx7aG8EwPnLZVzCCws"

In [ ]:
from roboflow import Roboflow
from configs.paths.PathConfig.ML.Detection.PATH_TO_SAVE_DATASET

rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("tree-detection-v4x2j").project("tree-quality-3")
version = project.version(7)
dataset = version.download("yolov11", location=PATH_TO_SAVE_DATASET)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Tree-Quality-3-7 in yolov11:: 100%|██████████| 1748/1748 [00:00<00:00, 5947.78it/s]
